#Data cleaning

Data cleaning steps:
- replace @ handles with: hdl (chose a word with no punctuation so no extra step is needed to process, hdl = handle)
- replace urls by: url
- replace emoticons with corresponding emojis


For all of these, first check what is the prevalence and if it is worth the effort. 

Ideas:
- split hashtags into words
    - Some people seperate words in hashtags with capitals. Make it much easier to seperate. 
- replace contractions
- spellchecking
- what does quoted messages mean? Someone quoting somebody else? Should we consider this text or remove it? 
- there are some smileys that are still in punctuation and not in unicode, eg :-P, :-)
- Tweets not in english
- Character ngram will probably be more efficient due to the really low quality of speach
- Retweets have two formats:
    - Either finish with RT &lt;content of retweet>
    - OR "&lt;content of retweet>" &lt;content of tweet>
    - Can also have mutliple embedings with “ for second level. E.g. : 
"@letwerkaaaaa: “@Palmira_0: HAHA WHEN I WAS LITTLE I WAS FAT AS FUCK:joy:” Same I was so fat that they thought my vagina was a dick" LMFAO
    - For now will leave them here. We might have to consider while training if it is retweeted content or original content. Actually impacts the single response rate, ie when people just add an emoji to a tweet (as in the emoji is the sole content) 

In [3]:
import json
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
pd.options.display.max_colwidth = 140
import nltk
import re

In [4]:
with open('./data/tweets_training.json','r') as f:
    tweets_df = pd.DataFrame(json.load(f))
clean_tweets_df = pd.DataFrame(tweets_df["text"])
# del tweets_df

### Replace @ handles with hdl

No "hdl" words for confusion in the txt. Good replacement name. 

Best to replace handles with "\ hdl\ ", so for tokenization it will be easier to identify as a word. 

In [5]:
pattern = r"@[a-zA-Z0-9_]{1,15}" #from http://kagan.mactane.org/blog/2009/09/22/what-characters-are-allowed-in-twitter-usernames/
print("{} handles replaced".format(np.sum(clean_tweets_df.text.str.contains(pattern).values)))
clean_tweets_df.text = clean_tweets_df.text.str.replace(pattern, " hdl ")
clean_tweets_df.sample(10)

293672 handles replaced


,text
676076,hdl I forgot to tell you I can't do it because I have soccer practice 😨
794937,I love the song Junk By Paul MCCartney ! Its so nice !
691481,hdl yes just let me know when you can!! Love you! 😘😘
249271,"I'm at Hillsdale Caltrain Station - hdl (San Mateo, CA) http://t.co/vKAJdDqPLY"
747414,hdl I died at this tweet
112751,hdl hdl hdl The great thing is that the stuff the Pokemon &amp; Sonic fanbases come up with makes that look sane and tasteful.
184431,500 words now time for a snack break and bathroom break lol
540456,"I'm not feeling that white smoke tho , we should of had black"
207285,If only
53473,Sharks player was giving me an autograph after the game and told me I was wearing the wrong team's jersey. Go hdl !


### Replace URLs with url

URLs are quiet well formed and are generally at the end of tweets. No risk of engulfing in the cleaning some more text after the url.

keyword url is used only 4 times in dataset, no risk of confusion

In [6]:
pattern = r"http://\S+"
print("{} urls replaced".format(np.sum(clean_tweets_df.text.str.contains(pattern).values)))
clean_tweets_df.text = clean_tweets_df.text.str.replace(pattern, " url ")
clean_tweets_df.sample(10)

182254 urls replaced


,text
591190,But idgaf lol
589389,The dumb stuff we do after studying in the library for 3 hours. @ J. Paul L. Library url
741933,hdl hdl I'm on it!
348670,I needa brush up on my math skills.
205760,"After years of getting bugged into making an Instagram, I finally made one ahahah. Follow me GioC415"
727679,I just woke Taylor up just to tell her we're going to BWW tonight
761952,Yeah me and Julissa skip class to come to valley fair because YOLO
161563,"hdl preordered it the second it was available, even tho I was in class ^.^ url"
68202,Girls varsity water polo at the half SF 7-VC 3
735805,"I'm at San Francisco, CA (San Francisco, CA) w/ 5 others url"


### Convert emoticons to emojis

In [7]:
# Based on:
# https://slack.zendesk.com/hc/en-us/articles/202931348-Emoji-and-emoticons
# http://unicodey.com/emoji-data/table.htm
# http://www.unicode.org/emoji/charts/emoji-list.html

emoticon2emoji = {
    r"<3": "\u2764",
    r"</3": "\U0001F494",
    r"8\)": "\U0001F60E",
    r"D:": "\U0001F627",
    r":'\(": "\U0001F622",
    r":o\)": "\U0001F435",
    r":-*\*": "\U0001F48B",
    r"=-*\)": "\U0001F600",
    r":-*D": "\U0001F600",
    r";-*\)": "\U0001F609",
    r":-*>": "\U0001F606",
    r":-*\|": "\U0001F610",
    r":-*[Oo]": "\U0001F62E",
    r">:-*\(": "\U0001F620",
    r":-*\)|\(:": "\U0001F603",
    r":-*\(|\):": "\U0001F61E",
    r":-*[/\\]": "\U0001F615",
    r":-*[PpbB]": "\U0001F61B",
    r";-*[PpbB]": "\U0001F61C"
}

In [8]:
for emoticon in emoticon2emoji:
    print("{:10} -> {:>5}".format(emoticon, emoticon2emoji[emoticon]))

:'\(       ->     😢
:-*\*      ->     💋
:-*D       ->     😀
;-*\)      ->     😉
:-*\)|\(:  ->     😃
D:         ->     😧
:-*[PpbB]  ->     😛
:-*[Oo]    ->     😮
>:-*\(     ->     😠
<3         ->     ❤
8\)        ->     😎
:-*\|      ->     😐
</3        ->     💔
:-*[/\\]   ->     😕
;-*[PpbB]  ->     😜
:o\)       ->     🐵
:-*\(|\):  ->     😞
:-*>       ->     😆
=-*\)      ->     😀


In [9]:
total = 0
for emoticon in emoticon2emoji:
    nreplacements = np.sum(clean_tweets_df.text.str.contains(emoticon).values)
    total += nreplacements
    print("{:3} replaced {:6} times".format(emoticon2emoji[emoticon], nreplacements))
    clean_tweets_df.text = clean_tweets_df.text.str.replace(emoticon, emoticon2emoji[emoticon])
print("{:3} replaced {} times".format("ALL", total))

😢   replaced    264 times
💋   replaced    236 times
😀   replaced   1221 times
😉   replaced   2689 times
😃   replaced  12555 times
😧   replaced    161 times
😛   replaced    961 times
😮   replaced    295 times
😠   replaced      0 times
❤   replaced      0 times
😎   replaced     84 times
😐   replaced     70 times
💔   replaced      0 times
😕   replaced   5939 times
😜   replaced    263 times
🐵   replaced      0 times
😞   replaced   4557 times
😆   replaced      0 times
😀   replaced    176 times
ALL replaced 29471 times


### Explore retweets

In [10]:
pattern = r"""(?:\W|^)RT(?:[ \":“]|$)| # Retweets with RT keyword
            [\"“]\s*hdl                # Retweets with quotes"""
temp = clean_tweets_df.text.str.contains(pattern, flags=re.X)
print("There are {} retweets".format(np.sum(temp.values)))
clean_tweets_df[temp].sample(10)

There are 40126 retweets


,text
518928,😍“ hdl : PERFECT!!! url
711525,RT hdl : Crochet Storage Basket Pattern: Free and Easy url via hdl
273890,"LIFE RT hdl : Gator boots👢 with a pimped out Gucci suit👔 ain't got no jawb💼, but I stay sharp🔪 #StillFly"
486359,""" hdl : “ hdl : CONCORD!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!” Yeeeeeee"" is full of RAIDER FANS, CUHZ THEY KNO WSSUP!! OAKLAND!!!!! Cuhah"
205832,“ hdl : I love how LAURA MARAVILLA can tweet but not reply” 👏👏👏👏👏👏👏👏
97106,Lmao at Melos RT rn
606491,“ hdl : I fainted upon entry to the new pink store.” It's time for an intervention
509515,!! 💃RT hdl The amount of time I've spent discussing the Phillip Lim for Target collection with hdl is sick.
468119,“ hdl : I was welcomed home by the best!💜 ΑΦ #latepost #lovemynewhome url you're the cutest 😘 AOE
727469,""" hdl : #InitialsOfSomeoneSpecial .. url \nSO Fucking true amen"


###Twitter hashtags